In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

# 1> Preprocessing

In [3]:
data=pd.read_csv("diabetes.csv")

In [4]:
display(data.info(),
        data.shape,
        data.size,
        data.describe(),
        data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


None

(768, 9)

6912

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
def remove_outliers(data):
    df_clean = data.copy()
    initial_rows = len(df_clean)
    
    for col in df_clean.select_dtypes(exclude=['object']).columns:
        Q1 = df_clean[col].quantile(0.15)
        Q3 = df_clean[col].quantile(0.85)
        IQR = Q3 - Q1
        df_clean = df_clean[(df_clean[col] >= Q1 - 1.5 * IQR) & 
                           (df_clean[col] <= Q3 + 1.5 * IQR)]
    
    removed = initial_rows - len(df_clean)
    display(f"Removed {removed} outliers ({removed/initial_rows:.2%} of data)")
    return df_clean
data=remove_outliers(data)

'Removed 65 outliers (8.46% of data)'

In [6]:
numeric_features = data.select_dtypes(exclude=['object']).columns
categorical_features = data.select_dtypes(include=['object']).columns

print("\nNumeric Features:", list(numeric_features))
print("Categorical Features:", list(categorical_features))


Numeric Features: ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
Categorical Features: []


In [7]:
x=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 703 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               703 non-null    int64  
 1   Glucose                   703 non-null    int64  
 2   BloodPressure             703 non-null    int64  
 3   SkinThickness             703 non-null    int64  
 4   Insulin                   703 non-null    int64  
 5   BMI                       703 non-null    float64
 6   DiabetesPedigreeFunction  703 non-null    float64
 7   Age                       703 non-null    int64  
 8   Outcome                   703 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.9 KB


In [10]:
for col in data.select_dtypes(include=['int64']).columns:
    data[col]=data[col].astype(np.int8)
for col in data.select_dtypes(include=['float64']).columns:
    data[col]=data[col].astype(np.float32)

In [11]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,-108,72,35,0,33.599998,0.627,50,1
1,1,85,66,29,0,26.600000,0.351,31,0
2,8,-73,64,0,0,23.299999,0.672,32,1
3,1,89,66,23,94,28.100000,0.167,21,0
5,5,116,74,0,0,25.600000,0.201,30,0


In [12]:
y.value_counts()

Outcome
0    469
1    234
Name: count, dtype: int64

In [13]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
smote = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
display(y_train.value_counts(),
y_resampled.value_counts())

Outcome
0    368
1    194
Name: count, dtype: int64

Outcome
0    368
1    368
Name: count, dtype: int64

# 2> Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=1)
model.fit(X_resampled, y_resampled)
model.accuracy = model.score(X_test, y_test)
display(model.accuracy)

0.6950354609929078

# 3> Random forest

In [15]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_resampled, y_resampled)
model.accuracy = model.score(X_test, y_test)
display(model.accuracy)

0.7304964539007093

# 4> Naive Bayes

In [23]:
from sklearn.naive_bayes import GaussianNB
#Used for features with continuous, numerical values.
model = GaussianNB()
model.fit(X_resampled, y_resampled)
model.accuracy = model.score(X_test, y_test)
display(model.accuracy)

0.7092198581560284

In [25]:
from sklearn.naive_bayes import BernoulliNB
#Used for binary features (0 or 1).
model = BernoulliNB(binarize=None)
model.fit(X_resampled, y_resampled)
model.accuracy = model.score(X_test, y_test)
display(model.accuracy)

C:\Users\kakan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\naive_bayes.py:1209: RuntimeWarning: invalid value encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


0.7163120567375887